In [15]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torchvision
import numpy as np
import matplotlib.pyplot as plt
from tensorboardX import SummaryWriter
# import scipy
# import StringIO

In [2]:
%matplotlib inline

In [3]:
batch_size = 32
learnning_rate = 1e-3
num_epochs = 3

In [4]:
train_dataset = datasets.MNIST('./datas', train=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST('./datas', train=False, transform=transforms.ToTensor())

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
class Cnn(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(Cnn, self).__init__()
        
        self.conv = nn.Sequential(
            nn.Conv2d(in_dim, 6, 3, stride=1, padding=1), # b 6 28 28
            nn.ReLU(True),
            nn.MaxPool2d(2, stride=2), # b 6 14 14
            nn.Conv2d(6, 16, 3, stride=1, padding=1), # b 16 14 14
            nn.ReLU(True),
            nn.MaxPool2d(2, 2), # b 16 7 7
            nn.Conv2d(16, 16, 3, stride=1), # b 32 5 5
            nn.ReLU(True),
        )
        
        self.fc = nn.Sequential(
            nn.Linear(400, 200),
            nn.Linear(200, 100),
            nn.Linear(100, out_dim),
        )
        
    def forward(self, x):
        out = self.conv(x)
        out = out.view(x.size(0), -1)
        return self.fc(out)
    

In [24]:
model = Cnn(1, 10).cuda()

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learnning_rate)

In [25]:
writer = SummaryWriter('./log/cnn')

In [26]:
for epoch in range(num_epochs):
    
    running_loss = .0
    running_acc = .0
    for i, data in enumerate(train_loader, 1):
        img, label = data
        img = img.cuda()
        label = label.cuda()
        
        out = model(img)
        loss = criterion(out, label)
        
        running_loss += loss.item() * img.size(0)
        
        _, pred = torch.max(out, 1)
        running_acc += (pred == label).sum().item()
        
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        step = epoch * len(train_loader) + i
        accuracy = (pred == label).float().mean()
        
        writer.add_scalar('loss', loss.item(), step)
        writer.add_scalar('accuracy', accuracy, step)
        writer.add_image('images', torchvision.utils.make_grid(img[:16]), step)
        
        model.conv.
        
        if i % 100 == 0:
            for tag, value in model.named_parameters():
                tag = tag.replace('.', '/')
                writer.add_histogram(tag, value.cpu().data.numpy(), step)
                writer.add_histogram(tag + '/grad', value.grad.cpu().data.numpy(), step)
        
        if i % 500 == 0:
            print 'Epoch: [{}/{}], Loss: {:.6f}, Acc: {:.6f}'.format(epoch + 1, num_epochs, \
                                                                    running_loss / (img.size(0) * i), \
                                                                    running_acc / (img.size(0) * i))
            
    print 'Finish {} Epoch, Loss: {:.6f}, Acc: {:.6f}'.format(epoch + 1, \
                                                             running_loss / len(train_dataset), \
                                                             running_acc / len(train_dataset))
        
    model.eval()
    eval_loss = .0
    eval_acc = .0
    for data in test_loader:
        img, label = data
        img = img.cuda()
        label = label.cuda()
        
        out = model(img)
        loss = criterion(out, label)
        
        eval_loss += loss.item() * img.size(0)
        
        _, pred = torch.max(out, 1)
        eval_acc += (pred == label).sum().item()
        
    print 'Eval Loss: {:.6f}, Eval Acc: {:.6f}'.format(eval_loss / len(test_dataset), eval_acc / len(test_dataset))

Epoch: [1/3], Loss: 0.439303, Acc: 0.855125
Epoch: [1/3], Loss: 0.281763, Acc: 0.908188
Epoch: [1/3], Loss: 0.221142, Acc: 0.928667
Finish 1 Epoch, Loss: 0.193993, Acc: 0.937417
Eval Loss: 0.056701, Eval Acc: 0.981100
Epoch: [2/3], Loss: 0.069399, Acc: 0.978625
Epoch: [2/3], Loss: 0.067727, Acc: 0.979031
Epoch: [2/3], Loss: 0.066395, Acc: 0.979688
Finish 2 Epoch, Loss: 0.064901, Acc: 0.979867
Eval Loss: 0.039565, Eval Acc: 0.987900
Epoch: [3/3], Loss: 0.052905, Acc: 0.983000
Epoch: [3/3], Loss: 0.053167, Acc: 0.983187
Epoch: [3/3], Loss: 0.053199, Acc: 0.983250
Finish 3 Epoch, Loss: 0.052645, Acc: 0.983383
Eval Loss: 0.057016, Eval Acc: 0.981400


In [27]:
writer.close()

In [ ]:
torch.save(model.state_dict(), './ser/cnn.pth')